In [1]:
import pathlib
from glob import glob
from keras.preprocessing import image

!ls -l ./img

total 24
drwxr-xr-x 2 lab11 multi 12288 Apr 16 15:53 0
drwxr-xr-x 2 lab11 multi 12288 Apr 16 15:53 1


In [6]:
163/2

81.5

In [16]:
from keras.preprocessing.image import ImageDataGenerator

# image directory
image_dir = './img_adj/img'

random_seed = 414

datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.3)

train_generator = datagen.flow_from_directory(image_dir,target_size = (640, 640), seed=random_seed,
                                                    batch_size = 81, class_mode = 'binary', subset='training')
valid_generator = datagen.flow_from_directory(image_dir, target_size=(640, 640), seed=random_seed,
                                              batch_size=81, class_mode='binary', subset='validation')


Found 382 images belonging to 2 classes.
Found 163 images belonging to 2 classes.


In [3]:
from tensorflow.keras.applications import ResNet50

conv_base =  ResNet50(weights='imagenet', include_top=False, input_shape =(640,640,3))

In [4]:
set_trainable = False

for layer in conv_base.layers:
    if layer.name =='conv5_block3_2_conv' :      
        set_trainable = True

    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

conv_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 640, 640, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 646, 646, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 320, 320, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 320, 320, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [13]:
from keras import layers, models, optimizers

model = models.Sequential()
model.add(conv_base)
model.add(layers.MaxPooling2D((3,3)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid')) 

model.summary()

model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=0.000005), metrics=['accuracy'])    

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 20, 20, 2048)      23587712  
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 2048)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 73728)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               9437312   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 33,025,153
Trainable params: 12,852,993
Non-trainable params: 20,172,160
_________________________________

In [14]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.33
session = InteractiveSession(config = config)

In [18]:
382//81

4

In [20]:
! time CUDA_VISIBLE_DEVICES=2 python tf_cnn_benchmarks.py --batch_size=1 --num_batches=30 --model=googlenet  --num_gpus=1 --lms=True --image_size=8192

python: can't open file 'tf_cnn_benchmarks.py': [Errno 2] No such file or directory

real	0m0.072s
user	0m0.000s
sys	0m0.017s


In [21]:
%%time

Hist = model.fit(train_generator, steps_per_epoch=382//81,
                epochs=50, validation_data=valid_generator, validation_steps=163//81) 

Epoch 1/50


ResourceExhaustedError:  SameWorkerRecvDone unable to allocate output tensor. Key: /job:localhost/replica:0/task:0/device:CPU:0;c4696a5a3bd9d747;/job:localhost/replica:0/task:0/device:GPU:0;edge_333_IteratorGetNext;0:0
	 [[{{node IteratorGetNext/_4}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_16925]

Function call stack:
train_function


In [ ]:
loss, accuracy = model.evaluate(test_generator, steps=5)
loss, accuracy

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize = (9, 6))
plt.plot(epochs, Hist.history['loss'])
plt.plot(epochs, Hist.history['val_loss'])

plt.legend(['Training Loss', 'Validation Loss'])
plt.grid()
plt.show()


plt.figure(figsize = (9, 6))
plt.plot(epochs, Hist.history['accuracy'])
plt.plot(epochs, Hist.history['val_accuracy'])

plt.legend(['Training accuracy', 'Validation accuracy'])
plt.grid()
plt.show()